The bug is that position continuity constraints don't seem to be enforced in my contact_graph gcs. I'm going to start by making the simplest case most vanilla gcs version.

In [1]:
from pydrake.all import (
    GraphOfConvexSets,
    GraphOfConvexSetsOptions,
    Cost,
    Constraint,
    Binding,
    MathematicalProgramResult,
    Point as DrakePoint,
    HPolyhedron,
    VPolytope,
    L2NormCost,
    eq,
)
import numpy as np

In [2]:
gcs = GraphOfConvexSets()
base_dim = 2
n_pos_per_set = 2
scaling = 10
source_set = DrakePoint(-3.56 * np.ones(base_dim * n_pos_per_set))
A_poly = np.vstack(
    (-np.eye(base_dim * n_pos_per_set), np.eye(base_dim * n_pos_per_set))
)
b_poly = np.ones(base_dim * n_pos_per_set * 2) * scaling

mid_set = HPolyhedron(A_poly, b_poly)
target_set = DrakePoint(1.08234 * np.ones(base_dim * n_pos_per_set))
gcs_edges = []
v_t = gcs.AddVertex(target_set, "t")
v_s = gcs.AddVertex(source_set, "s")
v_m = gcs.AddVertex(mid_set, "m")
edges = [(v_s, v_m), (v_m, v_t)]

# Add vertex cost
A = np.array([[-1, 1, 0, 0], [0, 0, -1, 1]])
b = np.array([0, 0])
vertex_path_length_cost = L2NormCost(A, b)
binding = Binding[Cost](vertex_path_length_cost, v_m.x())
v_m.AddCost(binding)

for edge in edges:
    e = gcs.AddEdge(*edge)
    gcs_edges.append(e)
    # Add edge position continuity constraint
    xu, xv = e.xu(), e.xv()
    constraints = eq(xu[[1, 3]], xv[[0, 2]])
    for c in constraints:
        print(c)
        e.AddConstraint(c)

options = GraphOfConvexSetsOptions()
options.convex_relaxation = False

result = gcs.SolveShortestPath(
    v_s,
    v_t,
    options,
)
assert result.is_success()
print(result.get_optimal_cost())
print(result.GetSolution(v_m.x()))

(s(1) == m(0))
(s(3) == m(2))
(m(1) == t(0))
(m(3) == t(2))
6.565260189147114
[-3.56     1.08234 -3.56     1.08234]


That worked, lets do the same thing but with the linear equality constraint

In [3]:
from pydrake.all import LinearEqualityConstraint

gcs = GraphOfConvexSets()

gcs_edges = []
v_t = gcs.AddVertex(target_set, "t")
v_s = gcs.AddVertex(source_set, "s")
v_m = gcs.AddVertex(mid_set, "m")
edges = [(v_s, v_m), (v_m, v_t)]

# Add vertex cost
A = np.array([[-1, 1, 0, 0], [0, 0, -1, 1]])
b = np.array([0, 0])
vertex_path_length_cost = L2NormCost(A, b)
binding = Binding[Cost](vertex_path_length_cost, v_m.x())
v_m.AddCost(binding)

# Aeq = np.array([
#     [0,-1,0,0,1,0,0,0],
#     [0,0,0,-1,0,0,1,0],
# ])
Aeq = np.array(
    [
        [0, 1, 0, 0, -1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, -1, 0],
    ]
)
beq = np.array([0, 0])
position_continuity_constraint = LinearEqualityConstraint(Aeq, beq)
for edge in edges:
    e = gcs.AddEdge(*edge)
    gcs_edges.append(e)
    # Add edge position continuity constraint
    edge_vars = np.array([e.xu(), e.xv()]).flatten()
    binding = Binding[Constraint](position_continuity_constraint, edge_vars)
    e.AddConstraint(binding)

options = GraphOfConvexSetsOptions()
options.convex_relaxation = False

result = gcs.SolveShortestPath(
    v_s,
    v_t,
    options,
)
assert result.is_success()
print(result.get_optimal_cost())
print(result.GetSolution(v_m.x()))

6.565260189147114
[-3.56     1.08234 -3.56     1.08234]


Ok... that worked. so maybe the next step is to build up and use my Graph wrapper

In [4]:
from large_gcs.graph.graph import Graph, DefaultGraphCostsConstraints
from large_gcs.geometry.point import Point
from large_gcs.geometry.polyhedron import Polyhedron

In [5]:
sets = [
    Point(source_set.x()),
    Polyhedron(mid_set.A(), mid_set.b()),
    Point(target_set.x()),
]
vertex_names = ["s", "m", "t"]

G = Graph(
    DefaultGraphCostsConstraints(
        vertex_costs=[vertex_path_length_cost],
        edge_constraints=[position_continuity_constraint],
    )
)
G.add_vertices_from_sets(sets, names=vertex_names)
G.set_source("s")
G.set_target("t")

G.add_edges_from_vertex_names(us=["s", "m"], vs=["m", "t"])

sol = G.solve()
print(sol.path)

Could not compute center
[('s', array([-3.56, -3.56, -3.56, -3.56])), ('m', array([-3.56   ,  1.08234, -3.56   ,  1.08234])), ('t', array([1.08234, 1.08234, 1.08234, 1.08234]))]


With multiple sets

In [6]:
gcs = GraphOfConvexSets()
base_dim = 2
n_pos_per_set = 2
scaling = 10
source_set = DrakePoint(-2 * np.ones(base_dim * n_pos_per_set))
A_poly = np.vstack(
    (-np.eye(base_dim * n_pos_per_set), np.eye(base_dim * n_pos_per_set))
)
b_poly = np.ones(base_dim * n_pos_per_set * 2) * scaling

mid_set = HPolyhedron(A_poly, b_poly)
mid_set_2 = HPolyhedron(A_poly, b_poly * 2)
target_set = DrakePoint(2 * np.ones(base_dim * n_pos_per_set))
gcs_edges = []
v_t = gcs.AddVertex(target_set, "t")
v_s = gcs.AddVertex(source_set, "s")
v_m = gcs.AddVertex(mid_set, "m")
v_m2 = gcs.AddVertex(mid_set_2, "m2")

edges = [(v_s, v_m), (v_m, v_t), (v_m, v_m2), (v_m2, v_m)]

# Add vertex cost
A = np.array([[-1, 1, 0, 0], [0, 0, -1, 1]])
b = np.array([0, 0])
vertex_path_length_cost = L2NormCost(A, b)
binding = Binding[Cost](vertex_path_length_cost, v_m.x())
v_m.AddCost(binding)

for edge in edges:
    e = gcs.AddEdge(*edge)
    gcs_edges.append(e)
    # Add edge position continuity constraint
    xu, xv = e.xu(), e.xv()
    constraints = eq(xu[[1, 3]], xv[[0, 2]])
    for c in constraints:
        print(c)
        e.AddConstraint(c)

options = GraphOfConvexSetsOptions()
options.convex_relaxation = False

result = gcs.SolveShortestPath(
    v_s,
    v_t,
    options,
)
assert result.is_success()
print(result.get_optimal_cost())
print(result.GetSolution(v_m.x()))

(s(1) == m(0))
(s(3) == m(2))
(m(1) == t(0))
(m(3) == t(2))
(m(1) == m2(0))
(m(3) == m2(2))
(m2(1) == m(0))
(m2(3) == m(2))
5.656854249492381
[-2.  2. -2.  2.]


Using the sets in my example that are broken:

In [7]:
gcs = GraphOfConvexSets()
base_dim = 2
n_pos_per_set = 2
scaling = 10
source_set = DrakePoint(-2 * np.ones(base_dim * n_pos_per_set))

m1_A = [[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0]]
m1_b = [-0.23333045, -0.23333045]
m2_A = [[-0.70710678, 0.0, 0.70710678, 0.0], [0.0, -0.70710678, 0.0, 0.70710678]]
m2_b = [-1.24921309, -1.24921309]

sets = [
    DrakePoint([-1, -1, -1, -1]),
    HPolyhedron(m1_A, m1_b),
    HPolyhedron(m2_A, m2_b),
    DrakePoint([-1, -1, -0.5, -0.5]),
]

vertex_names = ["s", "m1", "m2", "t"]
gcs_vertices = {name: gcs.AddVertex(set, name) for name, set in zip(vertex_names, sets)}

# Add vertex cost
A = np.array([[-1, 1, 0, 0], [0, 0, -1, 1]])
b = np.array([0, 0])
vertex_path_length_cost = L2NormCost(A, b)
for v in gcs_vertices.values():
    binding = Binding[Cost](vertex_path_length_cost, v.x())
    v.AddCost(binding)

edges = {"s": ["m1"], "m1": ["m2", "t"], "m2": ["m1"]}

gcs_edges = []

for u, vs in edges.items():
    for v in vs:
        e = gcs.AddEdge(gcs_vertices[u], gcs_vertices[v])
        gcs_edges.append(e)
        # Add constant edge cost
        e.AddCost(1)
        # Add edge position continuity constraint
        xu, xv = e.xu(), e.xv()
        constraints = eq(xu[[1, 3]], xv[[0, 2]])
        for c in constraints:
            e.AddConstraint(c)

options = GraphOfConvexSetsOptions()
options.convex_relaxation = False

result = gcs.SolveShortestPath(
    gcs_vertices["s"],
    gcs_vertices["t"],
    options,
)

assert result.is_success()


def print_results(result, gcs):
    print(f"optimal cost {result.get_optimal_cost()}")
    for e in gcs.Edges():
        print(f"{e.u().name()} -> {e.v().name()}, flow: {result.GetSolution(e.phi())}")
    for v in gcs.Vertices():
        print(v.name(), result.GetSolution(v.x()))


print("Original graph solution:")
print_results(result, gcs)
print("This DOES NOT satisfy the edge constraint betwen s and m1, and m1 and t")

print("\nGraph without m2 solution:")

gcs.RemoveVertex(gcs_vertices["m2"])

result = gcs.SolveShortestPath(
    gcs_vertices["s"],
    gcs_vertices["t"],
    options,
)

assert result.is_success()
print_results(result, gcs)
print("This satisfies the edge constraint betwen s and m1, and m1 and t")

Original graph solution:
optimal cost 2.5000000009732206
s -> m1, flow: 1.0
m1 -> m2, flow: 0.0
m1 -> t, flow: 1.0
m2 -> m1, flow: 0.0
s [-1. -1. -1. -1.]
m1 [-1.47986541 -1.47986541 -1.47986541 -0.97986541]
m2 [nan nan nan nan]
t [-1.  -1.  -0.5 -0.5]
This DOES NOT satisfy the edge constraint betwen s and m1, and m1 and t

Graph without m2 solution:
optimal cost 2.5
s -> m1, flow: 1.0
m1 -> t, flow: 1.0
s [-1. -1. -1. -1.]
m1 [-1.  -1.  -1.  -0.5]
t [-1.  -1.  -0.5 -0.5]
This satisfies the edge constraint betwen s and m1, and m1 and t


your HPolyhedra are unbounded -- interior point solvers perform badly if any variable is unbounded (roughly, that's because IPMs rely on barrier functions that penalize points near the boundary; these become ill-defined when the feasible set is unbounded).
my guess is this is a solver edge case.
generally it's a good practice to have all your variables be bounded when solving constrained optimization problems (though it does depend on the solver; IPOPT actually has a neat little feature of cursing at you for having unbounded variables).
it's also practically valid -- pretty much all robotics problems are physically bounded.
when i modify the hpolyhedra by adding a large artificial bounding box, I get valid solutions

In [8]:
gcs = GraphOfConvexSets()
base_dim = 2
n_pos_per_set = 2
scaling = 10
source_set = DrakePoint(-2 * np.ones(base_dim * n_pos_per_set))

m1_A = [[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0]]
m1_b = [-0.23333045, -0.23333045]
m2_A = [[-0.70710678, 0.0, 0.70710678, 0.0], [0.0, -0.70710678, 0.0, 0.70710678]]
m2_b = [-1.24921309, -1.24921309]

sets = [
    DrakePoint([-1, -1, -1, -1]),
    HPolyhedron(m1_A, m1_b),
    HPolyhedron(m2_A, m2_b),
    DrakePoint([-1, -1, -0.5, -0.5]),
]

vertex_names = ["s", "m1", "m2", "t"]
gcs_vertices = {name: gcs.AddVertex(set, name) for name, set in zip(vertex_names, sets)}

# Add vertex cost
A = np.array([[-1, 1, 0, 0], [0, 0, -1, 1]])
b = np.array([0, 0])
vertex_path_length_cost = L2NormCost(A, b)
for v in gcs_vertices.values():
    binding = Binding[Cost](vertex_path_length_cost, v.x())
    v.AddCost(binding)

edges = {"s": ["m1"], "m1": ["m2", "t"], "m2": ["m1"]}

print(f"m1 is bounded: {gcs_vertices['m1'].set().IsBounded()}")
print(f"m2 is bounded: {gcs_vertices['m2'].set().IsBounded()}")

gcs_edges = []

for u, vs in edges.items():
    for v in vs:
        e = gcs.AddEdge(gcs_vertices[u], gcs_vertices[v])
        gcs_edges.append(e)
        # Add constant edge cost
        e.AddCost(1)
        # Add edge position continuity constraint
        xu, xv = e.xu(), e.xv()
        constraints = eq(xu[[1, 3]], xv[[0, 2]])
        for c in constraints:
            e.AddConstraint(c)

options = GraphOfConvexSetsOptions()
options.convex_relaxation = False

result = gcs.SolveShortestPath(
    gcs_vertices["s"],
    gcs_vertices["t"],
    options,
)

assert result.is_success()


def print_results(result, gcs):
    print(f"optimal cost {result.get_optimal_cost()}")
    for e in gcs.Edges():
        print(f"{e.u().name()} -> {e.v().name()}, flow: {result.GetSolution(e.phi())}")
    for v in gcs.Vertices():
        print(v.name(), result.GetSolution(v.x()))


print("Original graph solution:")
print_results(result, gcs)

m1 is bounded: False
m2 is bounded: False
Original graph solution:
optimal cost 2.5000000009732206
s -> m1, flow: 1.0
m1 -> m2, flow: 0.0
m1 -> t, flow: 1.0
m2 -> m1, flow: 0.0
s [-1. -1. -1. -1.]
m1 [-1.47986541 -1.47986541 -1.47986541 -0.97986541]
m2 [nan nan nan nan]
t [-1.  -1.  -0.5 -0.5]
